In [1]:
!pip install -q tensorflow-recommenders
!pip install -q scann
!pip install -q azure-storage-blob
!pip install -q tensorboard

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

2023-03-11 22:31:22.995346: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 22:31:23.713323: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 22:31:23.713415: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 22:31:23.713424: W tensorfl

In [3]:

# run main function
path = "/storage/aliccp/"
log_path = "/storage/aliccp/"

df_train = pd.read_parquet(path+'/train_processed/')

In [4]:
df_train.head(20)

,item_intention,item_brand,position,item_id,item_category,item_shop,user_item_categories,user_profile,user_group,user_id,...,user_intentions,user_brands,user_categories,user_shops,click,conversion,user_item_shops,user_item_brands,user_item_intentions,user_consumption
0,9093445,9154780,9351665,4186222,8316799,8416205,9355039,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,None,None,None,None
1,9104107,9188757,9351665,4186855,8316676,8482726,None,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,None,None,None,None
2,9115551,9165876,9351665,4479003,8316590,8542197,9354837,3438658,3438762,31390,...,3941161,3812616,449068,569621,1,0,9511769,9867762,10083008,None
3,9035395,9323908,9351665,4933915,8316177,8529759,9354448,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,None,None,10079838,None
4,9030554,9172179,9351665,4945663,8316589,8754197,9354836,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,None,None,None,None
5,9066808,9266968,9351665,5155477,8316590,8911529,9354837,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,None,None,10083008,None
6,9046231,9266968,9351665,5160945,8316590,8441502,9354837,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,None,None,10083008,None
7,9093406,9159151,9351665,5475829,8316856,8943621,9355093,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,9781962,9862825,None,None
8,9052961,9341635,9351665,5607875,8315295,8707840,None,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,9623119,None,None,None
9,9104804,9159151,9351665,6096793,8315276,8598958,9353608,3438658,3438762,31390,...,3941161,3812616,449068,569621,0,0,None,9862825,None,None


In [5]:

print(len(df_train))

#filter out long tail products and reduce size of dataset
df_train['counter'] = 1
df_agg = df_train[['item_id','counter']].groupby('item_id').count()
df_agg = df_agg.sort_values("counter",ascending=False)[:10000]
df_train = pd.merge(df_agg, df_train, on=['item_id', 'item_id'], how='left')
print(len(df_train))
df_train['counter'] = 1
cust_agg = df_train[['user_id','counter']].groupby('user_id').count()
cust_agg = cust_agg.sort_values("counter",ascending=False)[:8000]
df_train = pd.merge(cust_agg, df_train, on=['user_id', 'user_id'], how='left')
print(len(df_train))

41999771
9477639


/tmp/ipykernel_2940/1344724864.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'counter_y'} in the result is deprecated and will raise a MergeError in a future version.
  df_train = pd.merge(cust_agg, df_train, on=['user_id', 'user_id'], how='left')


1473399


In [6]:
train = df_train[['item_id','user_id','user_categories', 'user_item_categories',
                                       'user_intentions','user_item_intentions','user_shops','user_item_brands','click']]
features = ['item_id','user_id']
for i in features:
    train[i] = pd.to_numeric(train[i], errors='coerce',downcast="integer")
train= train.fillna(0)

print(len(train))

/tmp/ipykernel_2940/468427158.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i] = pd.to_numeric(train[i], errors='coerce',downcast="integer")


1473399


In [7]:

items = train['item_id'].unique()
items = tf.convert_to_tensor(items, dtype=tf.int64)

train = tf.convert_to_tensor(train, dtype=tf.int64)


train = tf.data.Dataset.from_tensor_slices(train)

ratings = train.map(lambda x: {
    "item_id": x[0], "user_id": x[1],"user_categories":x[2], "user_item_categories":x[3], 
    "user_intentions":x[4],"user_item_intentions":x[5], "user_shops":x[6], "user_item_brands":x[7],"click":x[8]})

tf.random.set_seed(42)
shuffled = ratings.shuffle(100000, seed=42, reshuffle_each_iteration=False)

split_record = round(len(train)*.8)
train = shuffled.take(split_record)
test = shuffled.skip(split_record).take(len(shuffled)-split_record)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
feature_names = ["item_id", "user_id","user_categories", "user_item_categories", 
    "user_intentions","user_item_intentions", "user_shops", "user_item_brands"]

vocabularies = {}

for feature_name in feature_names:
    print(feature_name)
    vocab = shuffled.batch(1_000_000).map(lambda x: x[feature_name])
    vocabularies[feature_name] = np.unique(np.concatenate(list(vocab)))
    


item_id
user_id
user_categories
user_item_categories
user_intentions
user_item_intentions
user_shops
user_item_brands


In [13]:
class DCN(tfrs.Model):

  def __init__(self, use_cross_layer, deep_layer_sizes, projection_dim=None):
    super().__init__()

    self.embedding_dimension = 32

    self._all_features = ["item_id", "user_id","user_categories", "user_item_categories", 
    "user_intentions","user_item_intentions", "user_shops", "user_item_brands"]

    self._embeddings = {}


    # Compute embeddings for int features.
    for feature_name in self._all_features:
      vocabulary = vocabularies[feature_name]
      self._embeddings[feature_name] = tf.keras.Sequential(
          [tf.keras.layers.IntegerLookup(
              vocabulary=vocabulary, mask_value=None),
           tf.keras.layers.Embedding(len(vocabulary) + 1,
                                     self.embedding_dimension)
    ])

    if use_cross_layer:
      self._cross_layer = tfrs.layers.dcn.Cross(
          projection_dim=projection_dim,
          kernel_initializer="glorot_uniform")
    else:
      self._cross_layer = None

    self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu")
      for layer_size in deep_layer_sizes]

    self._logit_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    #self.task = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.RootMeanSquaredError("RMSE")])
    self.task = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.BinaryCrossentropy("BCE")])

  def call(self, features):
    # Concatenate embeddings
    embeddings = []
    for feature_name in self._all_features:
      embedding_fn = self._embeddings[feature_name]
      embeddings.append(embedding_fn(features[feature_name]))

    x = tf.concat(embeddings, axis=1)

    # Build Cross Network
    if self._cross_layer is not None:
      x = self._cross_layer(x)

    # Build Deep Network
    for deep_layer in self._deep_layers:
      x = deep_layer(x)

    return self._logit_layer(x)

  def compute_loss(self, features, training=False):
    labels = features.pop("click")
    scores = self(features)
    return self.task(
        labels=labels,
        predictions=scores,
    )

In [14]:
cached_train = train.batch(8192).cache()
cached_test = test.batch(4096).cache()

In [15]:
epochs = 8
learning_rate = 0.1

In [16]:
model = DCN(use_cross_layer=True,
                deep_layer_sizes=[192, 192],
                projection_dim=None)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
#seems to be a bug
model.fit(cached_train, epochs=8, verbose=True)

Epoch 1/8
144/144 [==============================] - 123s 815ms/step - BCE: 0.4701 - loss: 0.0319 - regularization_loss: 0.0000e+00 - total_loss: 0.0319
Epoch 2/8
144/144 [==============================] - 1s 6ms/step - BCE: 0.4679 - loss: 0.0303 - regularization_loss: 0.0000e+00 - total_loss: 0.0303
Epoch 3/8
144/144 [==============================] - 1s 7ms/step - BCE: 0.4679 - loss: 0.0303 - regularization_loss: 0.0000e+00 - total_loss: 0.0303
Epoch 4/8
144/144 [==============================] - 1s 7ms/step - BCE: 0.4679 - loss: 0.0303 - regularization_loss: 0.0000e+00 - total_loss: 0.0303
Epoch 5/8
144/144 [==============================] - 1s 7ms/step - BCE: 0.4679 - loss: 0.0303 - regularization_loss: 0.0000e+00 - total_loss: 0.0303
Epoch 6/8
144/144 [==============================] - 1s 7ms/step - BCE: 0.4679 - loss: 0.0303 - regularization_loss: 0.0000e+00 - total_loss: 0.0303
Epoch 7/8
144/144 [==============================] - 1s 6ms/step - BCE: 0.4679 - loss: 0.0303 - regula

In [17]:
model.evaluate(cached_test)

72/72 [==============================] - 45s 117ms/step - BCE: 0.4943 - loss: 0.0320 - regularization_loss: 0.0000e+00 - total_loss: 0.0320


[0.4943443834781647, 0.03209109604358673, 0, 0.03209109604358673]

In [ ]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=vocabularies['user_id'], mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(vocabularies['user_id']) + 1, embedding_dimension)
])

item_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=vocabularies['item_id'], mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(vocabularies['item_id']) + 1, embedding_dimension)
])


In [ ]:
class KerasBaseDCN(tf.keras.Model):

    def __init__(self, user_model, item_model, deep_layer_sizes, projection_dim=None):
        super().__init__()
        self.item_model: tf.keras.Model = item_model
        self.user_model: tf.keras.Model = user_model


        self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu")
          for layer_size in deep_layer_sizes]

        self._logit_layer = tf.keras.layers.Dense(1, activation='sigmoid')
        
        self.task = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.BinaryCrossentropy("BCE")])

    def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

        # Set up a gradient tape to record gradients.
        with tf.GradientTape() as tape:

            # Loss computation.
            user_embeddings = self.user_model(features["user_id"])
            positive_item_embeddings = self.item_model(features["item_id"])
            x = tf.concat([user_embeddings,positive_item_embeddings], axis=1)
            # Build Deep Network
            for deep_layer in self._deep_layers:
                x = deep_layer(x)
            x = self._logit_layer(x)
            loss = self.task(predictions = x,labels = features['click'])

            # Handle regularization losses as well.
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

    def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

        # Loss computation.
        user_embeddings = self.user_model(features["user_id"])
        positive_item_embeddings = self.item_model(features["item_id"])
        x = tf.concat([user_embeddings,positive_item_embeddings], axis=1)
        # Build Deep Network
        for deep_layer in self._deep_layers:
            x = deep_layer(x)
        x = self._logit_layer(x)
        loss = self.task(predictions = x,labels = features['click'])

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics
    
model = KerasBaseDCN(user_model, item_model,deep_layer_sizes=[192, 192])
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

model.fit(cached_train, epochs=15)

Epoch 1/15
144/144 [==============================] - 53s 319ms/step - BCE: 0.1854 - loss: 0.0426 - regularization_loss: 0.0000e+00 - total_loss: 0.0426
Epoch 2/15
144/144 [==============================] - 0s 3ms/step - BCE: 0.1368 - loss: 0.0295 - regularization_loss: 0.0000e+00 - total_loss: 0.0295
Epoch 3/15
144/144 [==============================] - 0s 3ms/step - BCE: 0.1360 - loss: 0.0294 - regularization_loss: 0.0000e+00 - total_loss: 0.0294
Epoch 4/15
144/144 [==============================] - 0s 3ms/step - BCE: 0.1359 - loss: 0.0294 - regularization_loss: 0.0000e+00 - total_loss: 0.0294
Epoch 5/15
144/144 [==============================] - 0s 3ms/step - BCE: 0.1359 - loss: 0.0294 - regularization_loss: 0.0000e+00 - total_loss: 0.0294
Epoch 6/15
144/144 [==============================] - 0s 3ms/step - BCE: 0.1359 - loss: 0.0294 - regularization_loss: 0.0000e+00 - total_loss: 0.0294
Epoch 7/15
144/144 [==============================] - 0s 3ms/step - BCE: 0.1359 - loss: 0.0294 - 

In [ ]:
model.evaluate(cached_train)